In [ ]:
import pandas as pd
import json
import folium
import numpy as np
import re

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

from tqdm.notebook import tqdm

import time

from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

In [ ]:
ediya_driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
ediya_driver.maximize_window()
ediya_driver.get("https://www.ediya.com/contents/find_store.html#c")


In [ ]:
gu_list = [
    '강남구','강동구','강북구','강서구','관악구','광진구','구로구','금천구',
    '노원구','도봉구','동대문구','동작구','마포구','서대문구','서초구','성동구',
    '성북구','송파구','양천구','영등포구','용산구','은평구','종로구','중구','중랑구'
    ]

In [ ]:
ediya_stores = []

for gu  in tqdm(gu_list) :

    ediya_driver.find_element(By.XPATH,"""//*[@id="contentWrap"]/div[3]/div/div[1]/ul/li[2]/a""").click()
    time.sleep(1)
    ediya_driver.find_element(By.ID,"keyword").send_keys("서울 " + gu + Keys.ENTER)
    time.sleep(1) 

    html = ediya_driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    ediya_store = soup.find_all("li","item")

    for each in ediya_store :
        ediya_store_info = [
            gu, #re.search("\s\w+[ \S]*", each.text).group().split()[1],
            re.search("\w+ ", each.text).group()[:-1],
            re.search("\s\w+[ \S]*", each.text).group()[1:].split("(")[0],
            ''
        ]
        
        ediya_stores.append(ediya_store_info)

In [ ]:
for i in tqdm(range(len(ediya_stores))):
    lat_lng = geocoding(ediya_stores[i][2])
    ediya_stores[i][2] = lat_lng[0]
    ediya_stores[i][3] = lat_lng[1]

In [ ]:
ediya = pd.DataFrame(ediya_stores)
ediya.head()

In [ ]:
mapping = folium.Map(location=[37.53, 127], width="100%", height="100%", zoom_start=11.8)
    
for idx, row in ediya.iterrows():
    folium.Circle(
        location=[row[2], row[3]],
        radius=100,
        color='blue',
        fill=True,
        fill_color='blue'
        ).add_to(mapping)

mapping